In [41]:
import pandas as pd
import re
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import random


"""

refseq - ncbi-ftp/gene/DATA/gene2refseq.gz
pubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz
ensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz
human - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
mart - useast.ensembl.org/index.html
neighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz
orthologs - ncbi-ftp/gene/DATA/gene_orthologs.gz

"""

# refseq = pd.read_table('gene2refseq')
# pubmed = pd.read_table('gene2pubmed')
# ensembl = pd.read_table('gene2ensembl')
# mart = pd.read_csv('mart_export.txt',delimiter = ',')
# neighbors = pd.read_table('gene_neighbors')
# orthologs = pd.read_table('gene_orthologs')


'\n\nrefseq - ncbi-ftp/gene/DATA/gene2refseq.gz\npubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz\nensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz\nhuman - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz\nmart - useast.ensembl.org/index.html\nneighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz\northologs - ncbi-ftp/gene/DATA/gene_orthologs.gz\n\n'

In [3]:
human = pd.read_table('Homo_sapiens.gene_info')
aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep


In [4]:
def compare_sets(list_data):
    for ele in list_data:
        col_names = ele.columns
        print(col_names)
# data_list = refseq,pubmed,ensembl,human,mart,neighbors,orthologs
# compare_sets(data_list)

In [5]:
def combined_translate(input_file, file_type, trans_list):
    if (file_type == 'csv'):
        df = pd.read_csv(input_file)
    if (file_type == 'excel'):
        df = pd.read_excel(input_file)
    if (file_type == 'other'):
        df = pd.read_table(input_file)
    
    for item in trans_list:
        test_col = item[0]
        reference_df = item[1]
        reference_col = item[2]
        output_col = item[3]
        
        
        start_param = df[test_col].tolist()
        in_list = reference_df[reference_col].tolist()
        out_listoflist = []

        for i in range(len(start_param)):
            param = start_param[i]

            match_list=[]
            for j in range(len(in_list)):
                if str(param) in str(in_list[j]):
                    index_match = reference_df.iloc[j]
                    match_list.append(index_match)

            out_list = []
            for k in range(len(match_list)):
                if (output_col == 'MATCH'):
                    out_list.append(match_list[k][reference_col])
                else: 
                    out_list.append(match_list[k][output_col])

            out_listoflist.append(out_list)
        out_colname = 'output ' + reference_col
        df[out_colname] = out_listoflist

    
        nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
        matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
        nonmatches = df.iloc[nonmatches_ind]
        matches = df.iloc[matches_ind]
        
#         return str(round(100*len(nonmatches)/len(df),4))+' % mismatch '
        return round(100*len(nonmatches)/len(df),4)
#         print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",input_file, 'to')
    
#     print(df)
    
        
          

In [11]:
def match_lists(data_list):
    
    mismatch_list = []
    class_ids = []
    labels = []
    
    for i in range(len(data_list)):
        for j in range(len(data_list)):
#             if (i != j):
            ### translation 1 test col name
            test_col1 = data_list[i][1]

            ### translation 1 reference dataset (human,gene,mart)
            if (data_list[j][2] == 'csv'):
                ref_df1 = pd.read_csv(data_list[j][0])
            if (data_list[j][2] == 'excel'):
                ref_df1 = pd.read_excel(data_list[j][0])
            if (data_list[j][2] == 'other'):
                ref_df1 = pd.read_table(data_list[j][0])

            ### translation 1 reference col name
            ref_col1 = data_list[j][1]

            ### translation 1 output col name
            out_col1 = 'MATCH'

            trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
            match_percent = combined_translate(data_list[i][0], data_list[i][2], trans_list)
            mismatch_list.append(match_percent)
            class_ids.append(data_list[j][3])
#                 print(match_percent, data_list[i][0], 'to', data_list[j][0])

        labels.append(data_list[i][3])
    
    match_list = []
    for ele in mismatch_list:
        match_list.append(100-ele)
    match_array = np.array(match_list).reshape(15, 15)
    
    id_list = []
    for ele in class_ids:
        id_list.append(ele)
    
    return match_array,id_list,labels


### Rheumetoid Arthritis data

In [12]:
md_2018 = pd.read_excel('md_2018.xlsx')
# https://journals.lww.com/md-journal/Fulltext/2018/06010/Identification_of_key_genes_in_rheumatoid.86.aspx
# A total of 313 genes (DEGs) were identified to be differentially expressed between RA and NC samples

ra_loci = pd.read_csv('RA_loci.txt')
# https://academic.oup.com/view-large/27924154
# Genetic loci associated with susceptibility to RA

yamamoto = pd.read_table('Yamamoto.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4729856/
# RA susceptible genes

radb = pd.read_table('RADB.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4164886/
# Genes and genetic regions that have the strongest association with RA susceptibility. 

okada = pd.read_table('okada.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3944098/#SD2
# Novel rheumatoid arthritis risk loci identified by trans-ethnic GWAS meta-analysis in >100,000 subjects.





### t2 diabetes

In [13]:
dia_supp = pd.read_excel('NIHMS795012-supplement-supp_table20.xlsx', header=2)
dia_supp


#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570412/
#SNP coverage and T2D association for 222 candidate gene regions (-10 kb/+5 kb)
gaulton_candidates = pd.read_excel('db_gaulton_candidates.xlsx')

#Stage 1 T2D SNP association for 3,531 genotyped SNPs, sorted by pSNP
gaulton = pd.read_excel('db_gaulton.xlsx')
gaulton

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4377835/
# Genetic loci associated with risk of T2D.
prasad = pd.read_table('prasad_t2d.txt')

# Genetic loci associated with glycemic traits.
prasad_gly = pd.read_table('prasad_glycemic.txt')
prasad_gly

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5537262/
t2diacod = pd.read_table('t2diacod.txt')

### heart disease

In [14]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3319439/
# Genes of interest within or near associated interval (genetic loci mapped by GWAS for myocardial 
# infarction or coronary artery disease)
kathiresan = pd.read_table('kathiresan.txt')

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5266753/
# 29 GWAS studies are included that identified more than 150 genomic loci associated with CAD and AMI 
barth = pd.read_excel('NIHMS764306-supplement-2.xlsx')
barth

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3684247/
joehanes = pd.read_table('Joehanes.txt')
joehanes

# https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/30586722/
aragam = pd.read_table('Aragam.txt')

l1 = ['md_2018.xlsx', 'Gene Symbol','excel','ra']
l2 = ['ra_loci.txt','Gene','csv','ra']
l3 = ['Yamamoto.txt','Gene','other','ra']
l4 = ['RADB.txt','Gene','other','ra']
l5 = ['okada.txt','Gene','other','ra']
l6 = ['dia_supp.xlsx','Gene name', 'excel','dia']
l7 = ['db_gaulton_candidates.xlsx', 'Gene symbol(s)', 'excel','dia']
l8 = ['db_gaulton.xlsx', 'Gene symbol', 'excel','dia']
l9 = ['prasad_t2d.txt', 'Gene/Nearest Gene', 'other','dia']
l10 = ['prasad_glycemic.txt', 'Gene/Nearest Gene', 'other','dia']
l11 = ['t2diacod.txt', 'Gene', 'other','dia']
l12 = ['kathiresan.txt', 'Gene', 'other','hd']
l13 = ['NIHMS764306-supplement-2.xlsx', 'Reported Gene(s)', 'excel','hd']
l14 = ['Joehanes.txt', 'Gene Symbol', 'other','hd']
l15 = ['Aragam.txt', 'Gene', 'other','hd']

combined_datasets = [l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14,l15]

matchArray,idList,labelList = match_lists(combined_datasets)


In [15]:
from itertools import groupby
from itertools import islice 


def avg_classifiers(match_array, id_list,num_classifiers):
    
    count_dups = [sum(1 for _ in group) for _, group in groupby(id_list)]
    
    avg_listoflist = []
    for i in range(match_array.shape[0]):
#         for j in range(match_array.shape[1]):
#             print(id_list[j+i*match_array.shape[1]])
#             print(match_array[i,j])
#         print(match_array[i])
        sub_list = []
        for k in range(num_classifiers):
            
            sub_list.append(count_dups[k+i*num_classifiers])

            slice_input = iter(match_array[i]) 
        slice_output = [list(islice(slice_input, elem)) for elem in sub_list] 
        
        avg_output = []
        for item in slice_output:
            avg_output.append(sum(item)/len(item))
        avg_listoflist.append(avg_output)
    return np.array(avg_listoflist)

In [16]:
pd.DataFrame(matchArray)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,100.0000,3.1949,3.1949,0.9585,3.1949,3.5144,1.5974,1.5974,0.6390,0.0000,6.0703,0.3195,0.6390,0.3195,0.0000
1,9.4340,100.0000,84.9057,23.5849,85.8491,5.6604,5.6604,5.6604,3.7736,0.0000,0.9434,0.0000,0.9434,0.0000,0.0000
2,7.6190,80.9524,100.0000,19.0476,90.4762,5.7143,6.6667,6.6667,3.8095,0.0000,0.9524,0.0000,0.9524,0.0000,0.0000
3,4.7619,80.9524,85.7143,100.0000,85.7143,0.0000,4.7619,4.7619,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,8.0000,77.0000,86.0000,20.0000,100.0000,6.0000,7.0000,7.0000,4.0000,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000
5,1.7323,1.1024,0.9449,0.1575,0.7874,100.0000,1.8898,1.8898,14.8031,0.7874,0.7874,0.9449,2.5197,0.3150,0.4724
6,2.7778,4.1667,4.6296,0.9259,4.6296,7.4074,100.0000,96.7593,4.1667,1.8519,6.9444,0.9259,1.8519,0.0000,0.4630
7,4.7862,4.4747,5.0694,0.5947,5.0694,8.6095,99.9717,100.0000,3.2569,0.7647,5.6924,0.7647,1.8408,0.0000,0.6797
8,2.6144,5.2288,5.2288,0.0000,4.5752,71.2418,10.4575,10.4575,100.0000,1.9608,4.5752,0.6536,7.8431,0.0000,0.6536
9,0.0000,0.0000,0.0000,0.0000,0.0000,16.6667,12.5000,12.5000,25.0000,100.0000,0.0000,0.0000,4.1667,0.0000,0.0000


In [162]:
matches = pd.DataFrame(matchArray)
matches
averages = avg_classifiers(matchArray,idList,3)
col_names = ['RA','DIA','HD']
averages_df = pd.DataFrame(averages,columns=col_names)

averages_df['Classifier'] = labelList
averages_df=averages_df.replace('ra',0)
averages_df=averages_df.replace('dia',1)
averages_df=averages_df.replace('hd',2)
averages_df


,RA,DIA,HD,Classifier
0,22.10864,2.236417,0.319500,0
1,60.75474,3.616367,0.235850,0
2,59.61904,3.968267,0.238100,0
3,71.42858,1.587300,0.000000,0
4,58.20000,4.166667,0.250000,0
5,0.94490,20.026250,1.063000,1
6,3.42592,36.188283,0.810200,1
7,3.99888,36.382533,0.821300,1
8,3.52944,33.115467,2.287575,1
9,0.00000,27.777783,1.041675,1


In [183]:
def cost_func(X, y, theta, fit_intercept = True):
    if fit_intercept:
        intercept = np.ones((X.shape[0], 1))
        X = np.concatenate((intercept, X), axis=1)
    
    z = np.dot(X, theta)
    h = 1 / (1+np.exp(-z))
    c = -(1/len(y)) * np.sum(y*np.log(h)+(1-y)*np.log(1-h))
    return c

def predict(x, theta):
    
    ## add intercept
    intercept = np.ones((x.shape[0], 1))
    x_wintercept = np.concatenate((intercept, x), axis=1)
    
    x_dotproduct = np.dot(x_wintercept, theta)
    g_sigmoid = 1 / (1+np.exp(-x_dotproduct))
    return g_sigmoid

def log_reg(X, y, array_len, case_num):

    rand_sample = random.randint(0,array_len-1)
    testing_X = X[rand_sample:rand_sample+1,]
    testing_y = y[rand_sample]
    training_X = np.delete(X, rand_sample, axis=0)
    training_y = np.delete(y, rand_sample)

    lf = LogisticRegression(random_state = 0, solver='liblinear',multi_class = 'auto').fit(training_X, training_y)
    theta = np.append(lf.intercept_,lf.coef_)
    sklearn_cost = cost_func(training_X, training_y, theta)
    prob1 = predict(testing_X, theta)

    
    
    ## Predict which dataset prob1 belongs
    if (case_num == 1):
        print("rheumetoid arthritis vs dia")
        if (prob1 < 0.5):
            dataset = col_names[0]
        else: dataset = col_names[1]
            
        print("Testing number: ", rand_sample)
        print("Target = ",testing_y, labelList[rand_sample])
        print("Prediction: ", prob1, dataset)
        
    if (case_num == 2):
        print("dia vs heart disease")
        if (prob1 < 0.5):
            dataset = col_names[1]
        else: dataset = col_names[2]
        print("Testing number: ", rand_sample+5)
        print("Target = ",testing_y, labelList[rand_sample+5])
        print("Prediction: ", prob1, dataset)  
        
    if (case_num == 3):
        print("rheumetoid vs heart disease")
        if (prob1 < 0.5):
            dataset = col_names[0]
        else: dataset = col_names[2]
        if (rand_sample<5):
            print("Testing number: ", rand_sample)
            print("Target = ",testing_y, labelList[rand_sample])
        else: 
            print("Testing number: ", rand_sample+5)
            print("Target = ",testing_y, labelList[rand_sample+5])
        print("Prediction: ", prob1, dataset)  
        
    print("sklearn logreg coefficients: ",lf.coef_, "Intercept: ",lf.intercept_)
    print("sklearn final cost = ",sklearn_cost)

X1 = np.array(averages_df)[0:11,0:3]
y1 = np.array(averages_df)[0:11,3]
log_reg(X1, y1, 11, 1)
print(" ")
X2 = np.array(averages_df)[5:15,0:3]
y2 = np.array(averages_df)[5:15,3]
log_reg(X2,y2,10, 2)
print(" ")
# X3 = np.array(averages_df)[0:5, 11:14]
X3 = np.concatenate((np.array(averages_df)[0:5,0:3],np.array(averages_df)[11:15,0:3]), axis=0)
y3 = np.concatenate((np.array(averages_df)[0:5,3],np.array(averages_df)[11:15,3]), axis=0)
log_reg(X3,y3,9, 3)

rheumetoid arthritis vs dia
Testing number:  7
Target =  1.0 dia
Prediction:  [0.99984913] DIA
sklearn logreg coefficients:  [[-0.2270313   0.26629625  0.02078453]] Intercept:  [0.0011844]
sklearn final cost =  0.0025291476793626103
 
dia vs heart disease
Testing number:  8
Target =  1.0 dia
Prediction:  [0.00028125] DIA
sklearn logreg coefficients:  [[-0.02655955 -0.26043793  0.23830804]] Intercept:  [-0.00286458]
sklearn final cost =  1.0086434061053942
 
rheumetoid vs heart disease
Testing number:  10
Target =  2.0 dia
Prediction:  [0.99994254] HD
sklearn logreg coefficients:  [[-0.21077687 -0.00901372  0.22687348]] Intercept:  [-0.001985]
sklearn final cost =  -2.211760888168444


In [184]:
def combine_datasets(listoflist):
    
    combined_data=[]
    label_list = []
    
    for i in range(len(listoflist)):
        list1 = listoflist[i]
        combined_disease=[]
        colnames = []
        for j in range(len(list1)):
            dataset = list1[j][0]
            col_name = list1[j][1]
            gene_list = dataset[col_name].tolist()
#             print(len(gene_list))
        
            combined_disease.append([gene_list])
            colnames.append(col_name)
        combined_data.append(combined_disease)
    
        label = [i] * len(list1)
        label_list.append(label)
    flat_list = [item for sublist in combined_data for item in sublist]
    flat_labels = [item for sublist in label_list for item in sublist]
    
    df = pd.DataFrame(flat_list)
    df['disease_type'] = flat_labels
#     df.columns = colnames
        
    return df

ra_list = [[md_2018, 'Gene Symbol'],
[ra_loci, 'Gene'],
[yamamoto, 'Gene'],
[radb, 'Gene'],
[okada, 'Gene']]



t2d_list = [[dia_supp, 'Gene name'],
[gaulton_candidates, 'Gene symbol(s)'],
[gaulton, 'Gene symbol'],
[prasad, 'Gene/Nearest Gene'],
[prasad_gly, 'Gene/Nearest Gene'],
[t2diacod, 'Gene']]

heart_list = [[kathiresan, 'Gene'],
[barth, 'Reported Gene(s)'],
[joehanes, 'Gene Symbol'],
[aragam, 'Gene']]

gene_df = combine_datasets([ra_list,t2d_list,heart_list])
# t2d_df = combine_datasets(t2d_list)
# heart_df = combine_datasets(heart_list)



In [85]:
x = gene_df.iloc[:,0].values.transpose()
y = gene_df.iloc[:,1].values.transpose()
# y = np.array([y])

for i in range(gene_df.shape[0]):
    x_train = np.delete(x, i)
    y_train = np.delete(y, i)
